#### Please read the text in each of the notebook cells. <BR>There are instructions in some of them that require input from you so that the code will run as expected.

### Data Sources
- Databricks can connect to source data from a variety of differnt storage mediums and formats
   - RDBMS
   - Message Bus (e.g. PubSub, Kafka, and more)
   - NoSQL 
   - Files (Delimited text, JSON, Avro, Parquet, ORC, Sequence Files, Images, Documents, and more)
   - Google Cloud Services: BigQuery, Bigtable, CloudSQL, PubSub, and other GCP services
   - In the cloud or on-premise, but Databricks is optimized for cloud object storage - Google Cloud Storage
- Connecting to Data Sources: https://docs.gcp.databricks.com/data/data-sources/index.html

### Authenticating to Google Cloud Storage in Databricks
Use a Service Account 
- Secure and does not expose any sensitive information
- Create a Service Account in Google Cloud, give that Service Account the proper permissions, associate to Google Cloud Storage bucket, <BR>and specify the Service Account used when launching the cluster. 
- There are other ways of using a Service Account instead of asscociating a Service Account to a cluster and you can add the Service Account key values into spark.conf. 
- Visit the online documentation to learn more about how to use Service Accounts with Databricks: https://docs.gcp.databricks.com/data/data-sources/google/gcs.html
  
<img src = "https://storage.googleapis.com/databricks-public-images/bmathew/service_account.png" height=600 width=600>

### Accessing Google Cloud Storage in Databricks
There are two common ways to access data in Google Cloud Storage from Databricks.
1. <b>Mount points</b>
   - Mounting your storage container to the Databricks workspace to be shared by all users and clusters.
   - This works well in a development environment where all users might need access to the data.
   - The mount is only a pointer to a GCS location and the data is never synced locally.
2. <b>No mount points</b>
   - Not using a mount point and instead using a specific service account that you have access to

Both methods require the use of a Service Account

### Using Mount Points
- Create a service account for the Databricks cluster.
   - The service account must be in the Google Cloud project that you used to set up the Databricks workspace.
- Configure the GCS bucket so that the service account has access to it.
- Launch a Databricks cluster with a service account attached to it.
- Note: In the example below, for demonstration purposes, we are using our user_id in the mount name; however, we would not typically do this in a live setting.
   - Make certain to replace the parameter "your-username" with your unique username
   - For example: odl_instructor_490106@databrickslabs.com --> odl_instructor_490106

In [0]:
%python
dbutils.fs.mount("gs://databricksgcplabs","/mnt/odl_user_768559/databricksgcplabs")

Out[1]: True

- Once mounted, we can view and navigate the contents of our GCS bucket using Databricks %fs file system commands.
- Databricks file system commands has similar Unix style syntax (e.g. ls, head, cp, rm, and mkdirs - Unix is actually mkdir)
- Replace the parameter "your-username" with your unique username

In [0]:
%fs
ls /mnt/odl_user_768559/databricksgcplabs

path,name,size
dbfs:/mnt/odl_user_768559/databricksgcplabs/atm_dataset/,atm_dataset/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/clickstream-json/,clickstream-json/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/members/,members/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/merge-data/,merge-data/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/my_custom_functions-0.1-py3.6_2022.egg,my_custom_functions-0.1-py3.6_2022.egg,2029
dbfs:/mnt/odl_user_768559/databricksgcplabs/products/,products/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/sales_data/,sales_data/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/tmp/,tmp/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/tmp1/,tmp1/,0
dbfs:/mnt/odl_user_768559/databricksgcplabs/transactions/,transactions/,0


- Unmount the mount point
- Replace the parameter "your-username" with your unique username

In [0]:
%python
dbutils.fs.unmount("/mnt/odl_user_768559/databricksgcplabs")

/mnt/odl_user_768559/databricksgcplabs has been unmounted.
Out[2]: True

### No Mount Points
- Create a service account for the Databricks cluster.
   - The service account must be in the Google Cloud project that you used to set up the Databricks workspace.
- Configure the GCS bucket so that the service account has access to it
- Launch a Databricks cluster with a service account attached to it

- Use Databricks %fs file system commands directly against the GCS bucket

In [0]:
%fs
ls gs://databricksgcplabs

path,name,size
gs://databricksgcplabs/tmp/,tmp/,0
gs://databricksgcplabs/tmp1/,tmp1/,0
gs://databricksgcplabs/members/,members/,0
gs://databricksgcplabs/products/,products/,0
gs://databricksgcplabs/user_logs/,user_logs/,0
gs://databricksgcplabs/merge-data/,merge-data/,0
gs://databricksgcplabs/sales_data/,sales_data/,0
gs://databricksgcplabs/atm_dataset/,atm_dataset/,0
gs://databricksgcplabs/transactions/,transactions/,0
gs://databricksgcplabs/clickstream-json/,clickstream-json/,0


In [0]:
%fs
head gs://databricksgcplabs/members/members.csv

[Truncated to first 65536 bytes]
msno,city,bd,gender,registered_via,registration_init_time
Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,,11,20110911
+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,,7,20110914
cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,,11,20110915
9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,,11,20110915
WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915
yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916
jNCGK78YkTyId3H3wFavcBLDmz7pfqlvCfUKf4G1Lw4=,1,0,,7,20110916
WH5Jq4mgtfUFXh2yz+HrcTXKS4Oess4k4W3qKolAeb0=,5,34,male,9,20110916
tKmbR4X5VXjHmxERrckawEMZ4znVy1lAQIR1vV5rdNk=,5,19,male,9,20110917
I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918
OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,0,,7,20110918
dCvvBHlaOAqgkAcv36IZ9twaCyH0Z9om0wEiKn13B3o=,22,18,male,9,20110919
6bra2AiVV8SGlm7R6mp97Ot5jU9FBH2/8JaRk0iaKco=,4,34,female,9,20110919
4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,28,female,9,20110920
iOzdu4IHbJxhB5CPpiQsZcGdptCm7OLwWm1Ci6oHLSE=,12,29,female,9,20110922
hZia/3iyvtThD1kv6ozRjRj8T0CAz0+zUFxs3e27wo8=,1,0,,9,20110922
QlDflP89J5KChheo2sbtH++SSDY/hUDj54gkgdfE76U=,1,0,,9,20110922
fOpj0ApSXk1NVODvC5db5Tvm/1G6exXxgTiCwTqvQn0=,13,31,female,7,20110923
GsPgdrXoV6688/0RFAwjKxQUpBB28JxK/Nx45kEpH60=,1,0,,7,20110924
QHHKGOY5yLrGdR8WYb0JtdJTP36Ii1QP3QTXxjEpGLk=,5,19,female,9,20110925
SpiCVwYrNY2XIpPpZkHHq4LDTfYyPNxBx7e+klc19H4=,15,22,female,9,20110925
+N2Na8js4BYLhkDXYD94o/5Tn5j1yi9u/+6F8an1MUs=,4,23,male,9,20110925
F6F2FJSHqQ/TLf7fQHJe1nbonEZmrCHtzG87U8VD9rE=,11,22,female,9,20110925
FotY8kXSC6tIyCuVn0NoaXjo0Hd854wuoNMWuwHaLIA=,5,29,male,9,20110926
glvtik6LgKzz/VHI8f7dAAHW4iqU8azER8mEsImDe78=,9,21,male,9,20110926
x+j0IrKnXLIqZawwAtwldq2by7TLMKyohbtSeT5gtDg=,9,22,male,9,20110926
7XXoaLnR+58yBjV/dtpvsvnkbadWYxO3+KkLgaaJ0VM=,5,28,male,7,20110927
GqYHRxlZChiZvB1uzR410wcQzuxqZNZci4AzOTzkAao=,5,27,male,9,20110927
vqz848bjeylg9nGoax8X4qQX2IX8qPwEq+4vbOpgy4c=,13,47,female,7,20110928
ZXtW2HuuVK4aPiO10RhBWT1PmLV+bKqnFlT5bL6YhoQ=,4,0,,9,20110928
qRUnTIEIOU70qVRjrqV9kmp8NmiKcuWKuHcvbV5Hgvs=,5,34,male,9,20110928
Z6WIOK9vXy+e2XDBiioNAxuZ0ScXSU/Ebq4tUwqVSrE=,22,38,female,9,20110929
BvmcS7e/M1+dAXapzH/ze8LxbdY+FV/v82vUvp3smOc=,14,30,male,9,20110930
BVbyThi0+EcLz1vM5oK+oDubZDQ4P6OAQqV5kDub3+A=,4,31,male,9,20111001
ATvHpah6ryn3yubmGZIxakYDFXa40ZBvouideMm+3EI=,5,36,female,9,20111001
PytoSP34NteOlJ1jUZWLfzUCZuVxJgBlZ4hC/ggEhss=,1,0,male,7,20111002
HLthcFr7gfJaH3BWCe3Y8Rzkn35g6MrjQ0JVPPwqrOc=,6,26,female,9,20111004
den0Kb2s4BV47zV+tSC1u0W07M7BOMq+fnrGj+9ax0I=,14,26,female,9,20111005
i4kmzPli+nl4XagzznO+oCcPeXfjcLSKIn8xE9oGU5E=,4,58,male,9,20111006
XmV2kHCnqnQf4oTJ4LCS7F02cpxnzqGIDDRGHe4dw8c=,22,31,female,9,20111006
kCe9c7EAz6+ILhukNbyQ7QnATImCBAM5EClCYln1t+Y=,4,43,female,9,20111006
i3rM6cvgxhpWoRUyWpPHMYNmAGvyt6V+kiTL6KAkrLk=,5,36,male,9,20111008
Xu8NWVApL13fqXD8N1vGafCLxVSw/FSOgUaJo5EdehM=,13,0,,7,20111008
mcultqpt4jx8RZyTffrPLdVz5EHp/1StLyzAaJz7wI4=,4,33,female,9,20111008
Mbo6fZW12wtpuWILrNSiZszTc71Xj/kZ749chrXArAQ=,13,0,,9,20111008
RA5kFIcw5aItrnKLvNc3GNcU+tWYlCPXzI1UpT16oFk=,1,0,,9,20111009
hYG4uW0hGiNIPblEpvkBWJdQ05Y3X1lINdYBqK5jBc8=,11,19,male,9,20111009
75A0yqLXt/ZZ+DY30gjRr/mJG8rDOexVswfVw1dVbt0=,15,21,male,9,20111010
wrX9H4DCVEdjzgUerZQLmOqdETgcO4gwb5SYdWUrEqo=,15,31,female,9,20111010
0oNjf4wD4O2p3Epo2cNmHzkyRpe1kqnjTEE4SIUtlUY=,4,28,male,9,20111012
tlGR5F/+vDFsnQme+KnvXKsCh3y31I1/dBaGuHDfdII=,5,39,female,7,20111012
JTV4R7YuiLDcRUE6pGwYRsXrbVGudErtEaY9IUeBV+E=,12,31,male,9,20111012
gVW9O1DA4x79cOpHwbdUJQzxHmGOy1Fgk5YzGCwPOXw=,22,24,male,9,20111012
1aMGSauvi/8+ERbGqIpBrL0tUWx7TmK3+TEAP1+1npg=,1,0,,9,20111012
IskrDwZywYJkdXY8cMbbmMdEC5c0VLrwKDyhexbZ1DQ=,6,33,female,9,20111013
FUhKSrJoIzqN3Sdu8ddFDdDVS4nFVqoQeXf8QNuqTKQ=,6,26,male,9,20111013
9Ebkk8c69slEG4FQ9KMgOMbt+VXo+RbPJat1QVou91M=,22,31,male,9,20111014
jn/lbZ3oyU0QhViguMecdTZbmG49VQp3C8H3DXI70to=,5,24,male,9,20111014
lYrf6Y/NjJ8yDGp51Nb537ZZ3bjy2ZmvjCE6Qgj53NM=,1,0,,7,20111015
kS+GS/0OJasgUQAUeJHKNkKLNHAV3hm

- To run multiple commands in the same cell, use Databricks Utilities, or DBUtils, which executs File System commands
- Here is an example that will create a new directory, copy files to it, and them remove the directory and its contents
- The example below is doing a recursive copy so that any subdirectories will also get copied
- Replace the parameter "your-username" with your unique username

In [0]:
%python
dbutils.fs.mkdirs("/tmp/odl_user_768559/data")
dbutils.fs.cp("gs://databricksgcplabs/products","/tmp/odl_user_768559/data", recurse=True)
dbutils.fs.rm("/tmp/odl_user_768559/data", recurse=True)

Out[3]: True

### Now that you learned how to authenticate with data in GCS, let's complete the final lab and build a Data Lakehouse!

#### [Click here to return to agenda]($./Agenda)